# Monte Carlo Method with Exploring Starts

In [3]:
import numpy as np
import random

def generate_episode_from_exploring_starts(policy, grid, terminal_states):
    # Randomly choose an initial state and action from non-terminal states
    non_terminal_states = [state for state in policy.keys() if state not in terminal_states]
    start_state = random.choice(non_terminal_states)
    start_action = np.random.choice(list(policy[start_state].keys()), p=list(policy[start_state].values()))
    episode = [(start_state, start_action, 0)]  # reward for the first action is 0 as it's part of the setup
    current_state = start_state
    while current_state not in terminal_states:
        next_state, reward = step(current_state, start_action, grid, terminal_states)
        episode.append((next_state, start_action, reward))
        if next_state in terminal_states:
            break
        current_state = next_state
        start_action = np.random.choice(list(policy[current_state].keys()), p=list(policy[current_state].values()))
    return episode

def step(state, action, grid, terminal_states):
    n = len(grid)
    x, y = state
    if action == 'U' and x > 0:
        x -= 1
    elif action == 'D' and x < n - 1:
        x += 1
    elif action == 'L' and y > 0:
        y -= 1
    elif action == 'R' and y < n - 1:
        y += 1
    next_state = (x, y)
    reward = -0.2 if next_state not in terminal_states and next_state != state else 0
    return next_state, reward

def mc_control_exploring_starts(grid, terminal_states, episodes, gamma=0.95):
    # Initialize policy as a distribution over actions for non-terminal states
    policy = {}
    Q = {}
    returns = {}
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if (i, j) not in terminal_states:
                actions = ['U', 'D', 'L', 'R']
                policy[(i, j)] = {action: 1.0 / len(actions) for action in actions}  # Uniform distribution initially
                Q[(i, j)] = {action: 0 for action in actions}
                returns[(i, j)] = {action: [] for action in actions}

    for _ in range(episodes):
        episode = generate_episode_from_exploring_starts(policy, grid, terminal_states)
        G = 0
        for state, action, reward in reversed(episode):
            if state not in terminal_states:
                G = gamma * G + reward
                if not any(x[0] == state and x[1] == action for x in episode[:-1]):
                    returns[state][action].append(G)
                    Q[state][action] = np.mean(returns[state][action])
                    best_action = max(Q[state], key=Q[state].get)
                    # Policy improvement
                    policy[state] = {a: 0.1 / len(actions) for a in actions}
                    policy[state][best_action] += 0.9

    return policy

# Define the grid and terminal states
grid = [[0] * 5 for _ in range(5)]  # Simplified grid representation
terminal_states = [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2)]  # Black squares as terminal states

# Run Monte Carlo Control with Exploring Starts
optimal_policy = mc_control_exploring_starts(grid, terminal_states, 10000)
print(optimal_policy)


{(0, 0): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (0, 1): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (0, 3): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (0, 4): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (1, 0): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (1, 1): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (1, 3): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (1, 4): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (2, 0): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (2, 1): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (2, 3): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (2, 4): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (3, 0): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (3, 1): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (3, 3): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (3, 4): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (4, 0): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (4, 1): {'U': 0.25, 'D': 0.25, 'L': 0.25, 'R': 0.25}, (4, 3): {'U': 0.25, 'D': 0.

# Monte Carlo with ε-soft Policy

In [5]:
import numpy as np
import random

def initialize_grid():
    # Create a 5x5 grid with specific states labeled
    grid = np.zeros((5, 5))
    terminal_states = [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2)]  # Black squares as terminal states
    special_rewards = {(0, 1): 5, (0, 3): 2.5, (1, 1): 0.5, (1, 3): -1}  # Blue, Green, Red, Yellow
    return grid, terminal_states, special_rewards

def step(state, action, terminal_states, special_rewards):
    x, y = state
    if action == 'U':
        x = max(0, x-1)
    elif action == 'D':
        x = min(4, x+1)
    elif action == 'L':
        y = max(0, y-1)
    elif action == 'R':
        y = min(4, y+1)
    
    next_state = (x, y)
    if next_state in terminal_states:
        reward = 0  # No reward for entering a terminal state
    elif next_state in special_rewards:
        reward = special_rewards[next_state]  # Special rewards for colored squares
    else:
        reward = -0.2  # Standard movement penalty

    return next_state, reward

def generate_episode(policy, grid, terminal_states, special_rewards):
    start_state = (2, 2)  # Start from the center of the grid or another non-terminal state
    episode = []
    state = start_state
    while state not in terminal_states:
        action = np.random.choice(['U', 'D', 'L', 'R'], p=policy[state])
        next_state, reward = step(state, action, terminal_states, special_rewards)
        episode.append((state, action, reward))
        state = next_state
    return episode

def mc_control_epsilon_soft(grid, terminal_states, special_rewards, episodes, epsilon=0.1, gamma=0.95):
    policy = {(i, j): np.array([0.25, 0.25, 0.25, 0.25]) for i in range(5) for j in range(5) if (i, j) not in terminal_states}
    Q = {(i, j): np.zeros(4) for i in range(5) for j in range(5) if (i, j) not in terminal_states}
    returns = {(i, j): [[] for _ in range(4)] for i in range(5) for j in range(5) if (i, j) not in terminal_states}

    for episode_num in range(episodes):
        episode = generate_episode(policy, grid, terminal_states, special_rewards)
        G = 0
        for state, action, reward in reversed(episode):
            G = gamma * G + reward
            action_index = ['U', 'D', 'L', 'R'].index(action)
            if (state, action_index) not in [(x[0], ['U', 'D', 'L', 'R'].index(x[1])) for x in episode[:episode.index((state, action, reward))]]:
                returns[state][action_index].append(G)
                Q[state][action_index] = np.mean(returns[state][action_index])
                best_action_index = np.argmax(Q[state])
                policy[state] = np.ones(4) * epsilon / 4
                policy[state][best_action_index] += (1 - epsilon)

    return policy

# Initialize the grid and related structures
grid, terminal_states, special_rewards = initialize_grid()
# Run the Monte Carlo control with ε-soft policy
optimal_policy = mc_control_epsilon_soft(grid, terminal_states, special_rewards, 10000)
print(optimal_policy)


{(0, 0): array([0.25, 0.25, 0.25, 0.25]), (0, 1): array([0.25, 0.25, 0.25, 0.25]), (0, 3): array([0.25, 0.25, 0.25, 0.25]), (0, 4): array([0.25, 0.25, 0.25, 0.25]), (1, 0): array([0.25, 0.25, 0.25, 0.25]), (1, 1): array([0.25, 0.25, 0.25, 0.25]), (1, 3): array([0.25, 0.25, 0.25, 0.25]), (1, 4): array([0.25, 0.25, 0.25, 0.25]), (2, 0): array([0.25, 0.25, 0.25, 0.25]), (2, 1): array([0.25, 0.25, 0.25, 0.25]), (2, 3): array([0.25, 0.25, 0.25, 0.25]), (2, 4): array([0.25, 0.25, 0.25, 0.25]), (3, 0): array([0.25, 0.25, 0.25, 0.25]), (3, 1): array([0.25, 0.25, 0.25, 0.25]), (3, 3): array([0.25, 0.25, 0.25, 0.25]), (3, 4): array([0.25, 0.25, 0.25, 0.25]), (4, 0): array([0.25, 0.25, 0.25, 0.25]), (4, 1): array([0.25, 0.25, 0.25, 0.25]), (4, 3): array([0.25, 0.25, 0.25, 0.25]), (4, 4): array([0.25, 0.25, 0.25, 0.25])}


: 